# Set-up

In [1]:
# Basic imports
import pandas as pd
from tqdm.notebook import tqdm
import os
from os.path import join

# Calendar imports
from datetime import datetime, timedelta
import calendar
import time

# Scraping imports
import requests

In [12]:
pd.options.mode.chained_assignment = None

# Functions

In [4]:
year_zone_system = {
    2018: 2012,
    2019: 2012,
    2020: 2012,
    2021: 2021,
    2022: 2021
}

In [5]:
def get_load_data(url, headers, date_from, date_to):
    """Gets Terna load data from date_from to date_to by making a get requests to url with headers"""
    params = {'dateFrom': date_from, 'dateTo': date_to}
    r = requests.get(url, headers=headers, params=params)
    if r.status_code != 200:
        print(r.text)
        r.raise_for_status()
    df = pd.DataFrame(r.json()['totalLoad'])
    return df

# Collect data

## Load data

### From API

In [6]:
# API account parameters
access_url = 'https://api.terna.it/transparency/oauth/accessToken'
API_KEY = '8fdqj4wqqn2y86sb7tm7sh6z'
SECRET = 'QDhNN8KfSa'
payload = f'grant_type=client_credentials&client_id={API_KEY}&client_secret={SECRET}'
access_headers = {'Content-Type': 'application/x-www-form-urlencoded'}

In [7]:
r_access = requests.post(access_url, headers=access_headers, data=payload)
r_access.raise_for_status()

time.sleep(1) # We need to sleep one second otherwise we get blocked
headers = {'Authorization': 'Bearer {}'.format(r_access.json()['access_token'])}

In [8]:
# Load endpoint URL and headers
url = 'https://api.terna.it/transparency/v1.0/gettotalload'
headers = {'Authorization': 'Bearer {}'.format(r_access.json()['access_token'])}

In [10]:
# years = [2023, 2022, 2021, 2020, 2019]
years = [2024]
months = range(1, 5)
# months = [1]

df_list = []

for year in tqdm(years):
    for month in tqdm(months):
        date_from = datetime(year, month, 1)
        _, days_in_month = calendar.monthrange(year, month)
        date_to = date_from + timedelta(days=days_in_month-1)
        date_from, date_to = date_from.strftime('%d/%m/%Y'), date_to.strftime('%d/%m/%Y')
        
        print(f"Fetching data from {date_from} to {date_to}...")
        try:
            time.sleep(1)
            df_list.append(get_load_data(url, headers, date_from, date_to))
        except Exception as e:
            print(e)
            r_access = requests.post(access_url, headers=access_headers, data=payload)
            headers = {'Authorization': 'Bearer {}'.format(r_access.json()['access_token'])}
            time.sleep(1)
            df_list.append(get_load_data(url, headers, date_from, date_to))     
print("Done")

df = pd.concat(df_list, ignore_index=True)
df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Fetching data from 01/01/2024 to 31/01/2024...
Fetching data from 01/02/2024 to 29/02/2024...
Fetching data from 01/03/2024 to 31/03/2024...
Fetching data from 01/04/2024 to 30/04/2024...
Done


,Date,Total_Load_MW,Forecast_Total_Load_MW,Bidding_Zone
0,2024-01-31 23:45:00,2783.13,2872.928,South
1,2024-01-31 23:45:00,1975.38,2039.116,Sicily
2,2024-01-31 23:45:00,859.54,887.273,Sardinia
3,2024-01-31 23:45:00,18599.491,19199.604,North
4,2024-01-31 23:45:00,32388,33433.001,Italy
...,...,...,...,...
92891,2024-04-01 00:00:00,12141.298,12147.433,North
92892,2024-04-01 00:00:00,23748,23760,Italy
92893,2024-04-01 00:00:00,4233.298,4235.437,Centre-South
92894,2024-04-01 00:00:00,2064.252,2065.295,Centre-North


In [18]:
# HOTFIX: Put separately fetched years together
# df_list_1 = df_list[:]
# df = pd.concat([df, pd.concat(df_list_1, ignore_index=True)], ignore_index=True)
# df

In [11]:
df['Date'] = df.Date.astype('datetime64[ns]')
df = df[df.Date.dt.minute == 0] # The original granularity of the data is the hour and 15-min interval values are just interpolated
df['Total_Load_MW'] = df['Total_Load_MW'].astype(float)
df.drop('Forecast_Total_Load_MW', axis=1, inplace=True)
df.sort_values('Date', inplace=True)
df

/var/folders/q5/hqwjttz50sg2x53x_jw4yz300000gn/T/ipykernel_25621/3529608277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total_Load_MW'] = df['Total_Load_MW'].astype(float)
/var/folders/q5/hqwjttz50sg2x53x_jw4yz300000gn/T/ipykernel_25621/3529608277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('Forecast_Total_Load_MW', axis=1, inplace=True)
/var/folders/q5/hqwjttz50sg2x53x_jw4yz300000gn/T/ipykernel_25621/3529608277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

,Date,Total_Load_MW,Bidding_Zone
23807,2024-01-01 00:00:00,1749.911,Sicily
23806,2024-01-01 00:00:00,814.730,Sardinia
23805,2024-01-01 00:00:00,11254.188,North
23804,2024-01-01 00:00:00,22508.000,Italy
23803,2024-01-01 00:00:00,4695.334,Centre-South
...,...,...,...
69882,2024-04-30 23:00:00,939.272,Sardinia
69881,2024-04-30 23:00:00,1958.606,Sicily
69880,2024-04-30 23:00:00,2237.048,South
69885,2024-04-30 23:00:00,5352.592,Centre-South


In [13]:
# Check that there are no duplicates
check_dup = df.groupby(['Date', 'Bidding_Zone']).size()
check_dup[check_dup>1]

Series([], dtype: int64)

In [14]:
dup_dates = check_dup[check_dup>1].reset_index().Date.unique()
df[df.Date.isin(dup_dates)].sort_values(['Date', 'Bidding_Zone'])

,Date,Total_Load_MW,Bidding_Zone


No idea on the origin of these duplicates, let's just drop them

In [15]:
df.drop_duplicates(inplace=True)
check_dup = df.groupby(['Date', 'Bidding_Zone']).size()
check_dup[check_dup>1]

Series([], dtype: int64)

In [16]:
df.drop_duplicates(subset=['Date', 'Bidding_Zone'], inplace=True)
check_dup = df.groupby(['Date', 'Bidding_Zone']).size()
check_dup[check_dup>1]

Series([], dtype: int64)

In [17]:
df = df.pivot(columns='Bidding_Zone', values='Total_Load_MW', index='Date')
df

Bidding_Zone,Calabria,Centre-North,Centre-South,Italy,North,Sardinia,Sicily,South
Date,,,,,,,,
2024-01-01 00:00:00,541.531,1954.746,4695.334,22508.000,11254.188,814.730,1749.911,1497.560
2024-01-01 01:00:00,552.160,1898.504,4672.788,21659.000,10748.550,787.019,1740.540,1259.439
2024-01-01 02:00:00,545.645,1794.185,4499.260,20614.000,10255.411,749.244,1700.652,1069.603
2024-01-01 03:00:00,457.126,1728.863,4159.235,19453.999,9666.547,723.450,1611.241,1107.537
2024-01-01 04:00:00,438.472,1654.957,3876.163,18551.000,9291.039,702.347,1524.640,1063.382
...,...,...,...,...,...,...,...,...
2024-04-30 19:00:00,634.805,3066.054,6111.586,35168.000,19557.944,994.013,2233.928,2569.670
2024-04-30 20:00:00,714.055,3140.296,6559.554,35829.001,19374.467,1059.234,2399.503,2581.892
2024-04-30 21:00:00,687.966,3047.052,6300.233,34906.999,18993.049,1007.929,2354.168,2516.602


In [18]:
df.to_csv('data/1_input/load/source_2024-05/load_2024-01_to_2024-04.csv')

In [33]:
years = df.index.year.unique()
for year in tqdm(years):
    df[df.index.year == year].to_csv(f'data/source_load/refresh_202402/load_{year}.csv')

  0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
# HOTFIX: Group together load 2019-2023 and 2018
df = pd.read_csv('data/1_input/load/source_2024-02/load_2019_to_2023.csv')
df_2018 = pd.read_csv('data/1_input/load/yearly/load_2018.csv')
df = pd.concat([df_2018, df], ignore_index=True).set_index('Date')
df

,Calabria,Centre-North,Centre-South,Italy,North,Sardinia,Sicily,South
Date,,,,,,,,
2018-01-01 00:00:00,NaN,2653.638,4468.169,24743.000,11662.759,888.752,2083.771,2985.911
2018-01-01 01:00:00,NaN,2523.413,4277.290,23797.000,11567.556,827.743,1850.558,2750.440
2018-01-01 02:00:00,NaN,2389.800,4012.677,22175.999,10747.364,790.148,1750.474,2485.536
2018-01-01 03:00:00,NaN,2286.362,3719.836,21042.000,10229.732,791.042,1645.808,2369.220
2018-01-01 04:00:00,NaN,2211.796,3476.088,20146.999,9866.849,768.386,1536.844,2287.036
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,791.814,2734.069,6110.321,31780.000,16288.086,1082.299,2464.776,2308.635
2023-12-31 20:00:00,757.054,2637.329,5856.124,30074.000,15265.633,1034.443,2335.105,2188.312
2023-12-31 21:00:00,674.156,2389.981,5376.828,27463.001,13917.284,958.273,2113.560,2032.919


In [22]:
df.to_csv('data/1_input/load/load.csv')

### From downloaded file

In [11]:
folder  = "/Users/guillaume/Library/CloudStorage/OneDrive-PolitecnicodiMilano/Share/Elena - Guillaume/Fabbisogno al quarto d'ora - Terna"

df_list = []
for file in os.listdir(folder):
    df = pd.read_excel(join(folder, file))
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)
df

,ANNOMESE,DATA_QUARTO,REGIONE,FABBISOGNO100
0,202101,2021-01-01 00:00:00,CAMPANIA,428.926800
1,202101,2021-01-01 00:00:00,SICILIA,506.598536
2,202101,2021-01-01 00:00:00,BASILICATA,75.278756
3,202101,2021-01-01 00:00:00,MARCHE,149.007881
4,202101,2021-01-01 00:00:00,CALABRIA,148.858752
...,...,...,...,...
4206715,201812,2018-12-31 23:45:00,FRIULI,159.485507
4206716,201812,2018-12-31 23:45:00,CAMPANIA,465.428558
4206717,201812,2018-12-31 23:45:00,LAZIO,522.667095
4206718,201812,2018-12-31 23:45:00,BASILICATA,97.513893


In [34]:
test_df = df[(df.DATA_QUARTO.dt.date == pd.to_datetime('2022-03-11').date()) & (df.REGIONE == 'MARCHE')]

In [37]:
df = df.sort_values(by=['REGIONE', 'DATA_QUARTO'])
df

,ANNOMESE,DATA_QUARTO,REGIONE,FABBISOGNO100
3505935,201801,2018-01-01 00:00:00,ABRUZZO,130.427209
3505940,201801,2018-01-01 00:15:00,ABRUZZO,128.281197
3505961,201801,2018-01-01 00:30:00,ABRUZZO,130.751181
3505997,201801,2018-01-01 00:45:00,ABRUZZO,130.888127
3506014,201801,2018-01-01 01:00:00,ABRUZZO,131.885886
...,...,...,...,...
2805024,202312,2023-12-31 22:45:00,VENETO,478.740481
2805049,202312,2023-12-31 23:00:00,VENETO,471.162627
2805061,202312,2023-12-31 23:15:00,VENETO,466.604402
2805082,202312,2023-12-31 23:30:00,VENETO,469.752546


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4206720 entries, 3505935 to 2805104
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ANNOMESE       int64         
 1   DATA_QUARTO    datetime64[ns]
 2   REGIONE        object        
 3   FABBISOGNO100  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 160.5+ MB


In [39]:
df['REGIONE'].value_counts()

REGIONE
ABRUZZO                210336
BASILICATA             210336
VALLE D'AOSTA          210336
UMBRIA                 210336
TRENTINO ALTO ADIGE    210336
TOSCANA                210336
SICILIA                210336
SARDEGNA               210336
PUGLIA                 210336
PIEMONTE               210336
MOLISE                 210336
MARCHE                 210336
LOMBARDIA              210336
LIGURIA                210336
LAZIO                  210336
FRIULI                 210336
EMILIA ROMAGNA         210336
CAMPANIA               210336
CALABRIA               210336
VENETO                 210336
Name: count, dtype: int64

## Consumption data

In [68]:
filenames = os.listdir('data/consumption')

cons_df_list = []

for name in sorted(filenames):
    year = int(name[:4])
    cons_df = pd.read_excel(join('data/consumption', name), skiprows=2)
    cons_df['year'] = year
    cons_df.drop('YoY settore', axis=1, inplace=True)
    cons_df_list.append(cons_df)

cons_df = pd.concat(cons_df_list, ignore_index=True)
cons_df

,Regione,Settore corretto,Sum of Consumo corretto,year
0,Veneto,Agricoltura,708.884874,2018
1,Veneto,Domestico,5595.518524,2018
2,Veneto,Industria,14741.332908,2018
3,Veneto,Servizi,9455.671888,2018
4,Valle d'Aosta,Agricoltura,5.323899,2018
...,...,...,...,...
395,Basilicata,Servizi,681.338094,2022
396,Abruzzo,Agricoltura,165.674637,2022
397,Abruzzo,Domestico,1258.266604,2022
398,Abruzzo,Industria,2788.733122,2022


In [102]:
# Read region zone mapping
region_zone = {}
for year in [2012, 2021]:
    region_zone_df = pd.read_excel('data/region_zone.xlsx', sheet_name=str(year))
    region_zone[year] = dict(zip(region_zone_df['Region'], region_zone_df['Zone']))
region_zone

{2012: {'Veneto': 'North',
  "Valle d'Aosta": 'North',
  'Umbria': 'Centre-North',
  'Trentino-Alto Adige': 'North',
  'Toscana': 'Centre-North',
  'Sicilia': 'Sicily',
  'Sardegna': 'Sardinia',
  'Puglia': 'South',
  'Piemonte': 'North',
  'Molise': 'South',
  'Marche': 'Centre-North',
  'Lombardia': 'North',
  'Liguria': 'North',
  'Lazio': 'Centre-South',
  'Friuli-Venezia Giulia': 'North',
  'Emilia-Romagna': 'North',
  'Campania': 'Centre-South',
  'Calabria': 'South',
  'Basilicata': 'South',
  'Abruzzo': 'Centre-South'},
 2021: {'Veneto': 'North',
  "Valle d'Aosta": 'North',
  'Umbria': 'Centre-South',
  'Trentino-Alto Adige': 'North',
  'Toscana': 'Centre-North',
  'Sicilia': 'Sicily',
  'Sardegna': 'Sardinia',
  'Puglia': 'South',
  'Piemonte': 'North',
  'Molise': 'South',
  'Marche': 'Centre-North',
  'Lombardia': 'North',
  'Liguria': 'North',
  'Lazio': 'Centre-South',
  'Friuli-Venezia Giulia': 'North',
  'Emilia-Romagna': 'North',
  'Campania': 'Centre-South',
  'Calabri

In [103]:
cons_df['Zone'] = cons_df.apply(lambda row: region_zone[year_zone_system[row.year]][row.Regione], axis=1)
cons_df

,Regione,Settore corretto,Sum of Consumo corretto,year,Zone
0,Veneto,Agricoltura,708.884874,2018,North
1,Veneto,Domestico,5595.518524,2018,North
2,Veneto,Industria,14741.332908,2018,North
3,Veneto,Servizi,9455.671888,2018,North
4,Valle d'Aosta,Agricoltura,5.323899,2018,North
...,...,...,...,...,...
395,Basilicata,Servizi,681.338094,2022,South
396,Abruzzo,Agricoltura,165.674637,2022,Centre-South
397,Abruzzo,Domestico,1258.266604,2022,Centre-South
398,Abruzzo,Industria,2788.733122,2022,Centre-South


In [105]:
cons_df.to_excel('data/consumption/consumption.xlsx', index=False)

<HR>

In [43]:
model = pd.read_csv('/Users/guillaume/Documents/GitHub/BSS/data/source_load/load_with_calendar.csv', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/guillaume/Documents/GitHub/BSS/data/source_load/load_with_calendar.csv'

In [ ]:
model.columns

Index(['Calabria', 'Centre-North', 'Centre-South', 'Italy', 'North',
       'Sardinia', 'Sicily', 'South', 'year', 'month_day', 'day', 'time',
       'weekday', 'weekofyear', 'monthofyear', 'hour', 'daytype'],
      dtype='object')

In [ ]:
df = df[model.columns]
df

,Calabria,Centre-North,Centre-South,Italy,North,Sardinia,Sicily,South,year,month_day,day,time,weekday,weekofyear,monthofyear,hour,daytype
Date,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,NaN,2660.928,4292.272,24370.000,11776.405,942.891,1876.794,2820.710,2019,2020-01-01 00:00:00,2019-01-01,2020-01-01 00:00:00,Tuesday,1,1,0,Holiday
2019-01-01 01:00:00,NaN,2567.076,4147.282,23599.001,11559.374,894.427,1836.644,2594.198,2019,2020-01-01 01:00:00,2019-01-01,2020-01-01 01:00:00,Tuesday,1,1,1,Holiday
2019-01-01 02:00:00,NaN,2458.020,3994.622,22501.000,10916.388,862.613,1759.223,2510.134,2019,2020-01-01 02:00:00,2019-01-01,2020-01-01 02:00:00,Tuesday,1,1,2,Holiday
2019-01-01 03:00:00,NaN,2335.024,3691.788,21038.000,10194.686,829.157,1650.097,2337.248,2019,2020-01-01 03:00:00,2019-01-01,2020-01-01 03:00:00,Tuesday,1,1,3,Holiday
2019-01-01 04:00:00,NaN,2238.983,3436.792,19870.999,9562.170,809.330,1579.195,2244.529,2019,2020-01-01 04:00:00,2019-01-01,2020-01-01 04:00:00,Tuesday,1,1,4,Holiday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,791.814,2734.069,6110.321,31780.000,16288.086,1082.299,2464.776,2308.635,2023,2020-12-31 19:00:00,2023-12-31,2020-01-01 19:00:00,Sunday,52,12,19,Holiday
2023-12-31 20:00:00,757.054,2637.329,5856.124,30074.000,15265.633,1034.443,2335.105,2188.312,2023,2020-12-31 20:00:00,2023-12-31,2020-01-01 20:00:00,Sunday,52,12,20,Holiday
2023-12-31 21:00:00,674.156,2389.981,5376.828,27463.001,13917.284,958.273,2113.560,2032.919,2023,2020-12-31 21:00:00,2023-12-31,2020-01-01 21:00:00,Sunday,52,12,21,Holiday


In [ ]:
df.to_csv('data/source_load/refresh_202402/load_2019_to_2023_with_calendar.csv')

In [ ]:
df[df.index.year == 2023].to_csv(f'data/source_load/refresh_202402/load_2023_with_calendar.csv')